In [ ]:
# 🚀 1. Inicialização do Spark
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQL-Kafka-Postgres") \
    .config("spark.jars", "/caminho/para/postgresql-42.7.1.jar") \  # ⚠️ Atualize com caminho correto
    .getOrCreate()


In [ ]:
# ✅ 2. Leitura do PostgreSQL com tabela e query reais

df_pg = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/postgres") \
    .option("dbtable", "(SELECT \"CompraManha\", \"VendaManha\", \"PUCompraManha\", \"PUVendaManha\", \"PUBaseManha\", \"Data_Vencimento\", \"Data_Base\", \"Tipo\", dt_update FROM public.dadostesouroipca) AS dados") \
    .option("user", "postgres") \
    .option("password", "postgres") \
    .load()

df_pg.show()
df_pg.createOrReplaceTempView("tabela_ipca")


In [ ]:
# 📊 3. Consulta SQL sobre dados do PostgreSQL
spark.sql("""
    SELECT Tipo, COUNT(*) AS total
    FROM tabela_ipca
    GROUP BY Tipo
""").show()


In [ ]:
# 📦 4. Leitura do Kafka
# Certifique-se de que o tópico e o bootstrap server estão corretos

df_kafka = spark.read \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "nome_do_topico") \
    .load()

# Convertendo o valor em string
df_kafka_str = df_kafka.selectExpr("CAST(value AS STRING)")
df_kafka_str.show()
df_kafka_str.createOrReplaceTempView("mensagens_kafka")


In [ ]:
# 🔍 5. Consulta SQL sobre dados do Kafka
spark.sql("""
    SELECT value, LENGTH(value) as tamanho
    FROM mensagens_kafka
""").show()


In [ ]:
# 🛑 6. Encerramento
spark.stop()
